# CREATE GALAXY DUST MASK

In [1]:
import healpy as hp
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.io import fits

## Create mask

In [2]:
if not os.path.exists("ris"):
    print("Creating 'ris' directory")
    os.makedirs("ris")

Load the HFI 353 GHz map from the FITS file downloaded from the Planck Legacy Archive (PLA).

In [3]:
hfi353 = hp.read_map("/mnt/d/Tesi/data/MilkyWay/HFI_SkyMap_353-psb_2048_R3.01_full.fits")

# Reduce the resolution of the map in order to save memory and computational time
#print("Changing resolution")
#hfi353 = hp.ud_grade(hfi353, 512)

NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT


/home/paolo/.miniconda3/envs/idp/lib/python3.6/site-packages/healpy/fitsfunc.py:371: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  warnings.warn("No INDXSCHM keyword in header file : " "assume {}".format(schm))


Ordering converted to RING


Make the mask.

In [5]:
# Change the following value to find a suitable mask
THRESHOLD = 0.009

hfi353N = np.array(hfi353)

# Rotate the map from Galactic to Ecliptic coordinates
print("Rotate")
rotator = hp.rotator.Rotator(coord=["G","E"])
hfi353N = rotator.rotate_map_pixel(hfi353N)

# Apply a smoothing filter to the map
print("Smoothing filter")
hfi353N = hp.smoothing(hfi353N, fwhm=np.deg2rad(1.0), verbose=False)

# Normalize the pixel values
print("Normalize pixel values")
hfi353N -= np.min(hfi353N)
hfi353N /= np.max(hfi353N)

# Clip the values
print("Make mask")
hfi353N[hfi353N <= THRESHOLD] = 0
hfi353N[hfi353N > THRESHOLD] = 1

# Save the map in a new file
print("Saving mask")
file_name = "ris/HFI_DustMask_" + str(THRESHOLD) + ".fits.gz"
if os.path.exists(file_name):
    os.remove(file_name)
hp.write_map(file_name, hfi353N, coord="E")

Rotate
Smoothing filter
Normalize pixel values
Make mask
Saving mask


## Plot data

Plot galaxy map without threshold, normalized as hist.

_**hist**: this method usually increases the global contrast of many images, especially when the usable data of the image is represented by close contrast values. Through this adjustment, the intensities can be better distributed on the histogram. This allows for areas of lower local contrast to gain a higher contrast. Histogram equalization
accomplishes this by effectively spreading out the most frequent intensity values._

In [9]:
hfi353 = hp.read_map("/mnt/d/Tesi/data/MilkyWay/HFI_SkyMap_353-psb_2048_R3.01_full.fits")

# Galactic coordinates
print("Drawing galactic map")
hp.mollview(hfi353, coord="G", norm="hist")
plt.savefig("ris/galactic_DustMap.png", dpi=800)
# Ecliptic coordinates
print("Drawing ecliptic map")
hp.mollview(hfi353, coord="GE", norm="hist")
plt.savefig("ris/ecliptic_DustMap.png", dpi=800)

NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT


/home/paolo/.miniconda3/envs/idp/lib/python3.6/site-packages/healpy/fitsfunc.py:371: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  warnings.warn("No INDXSCHM keyword in header file : " "assume {}".format(schm))


Ordering converted to RING
Drawing galactic map
Drawing ecliptic map


Plot galaxy map with threshold.

In [7]:
# Select desired threshold
THRESHOLD = 0.009

file_name = "ris/HFI_DustMask_" + str(THRESHOLD) + ".fits.gz"
dust_map = hp.read_map(file_name)

# Galactic coordinates
print("Drawing galactic mask")
hp.mollview(dust_map, coord="EG")
plt.savefig("ris/galactic_DustMask_" + str(THRESHOLD) + ".png", dpi=800)
# Ecliptic coordinates
print("Drawing ecliptic mask")
hp.mollview(dust_map, coord="E")
plt.savefig("ris/ecliptic_DustMask_" + str(THRESHOLD) + ".png", dpi=800)

NSIDE = 2048
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
Drawing galactic mask
Drawing ecliptic mask
